In [14]:
# from bokeh.layouts import column
# from bokeh.models import ColumnDataSource, CustomJS, Slider
# from bokeh.plotting import Figure, output_file, show
# from bokeh.io import output_notebook, show

# #output_file("js_on_change.html")
# output_notebook()

# x = [x*0.005 for x in range(0, 200)]
# y = x

# source = ColumnDataSource(data=dict(x=x, y=y))

# plot = Figure(plot_width=400, plot_height=400)
# plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

# callback = CustomJS(args=dict(source=source), code="""
#     var data = source.data;
#     var f = cb_obj.value
#     var x = data['x']
#     var y = data['y']
#     for (var i = 0; i < x.length; i++) {
#         y[i] = Math.pow(x[i], f)
#     }
#     source.change.emit();
# """)

# slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
# slider.js_on_change('value', callback)

# layout = column(slider, plot)

# show(layout)


import yaml
from bokeh.themes import Theme
from bokeh.layouts import column, row
from bokeh.models import Select
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource

output_file("js_on_change.html")

import pandas as pd

df = df.copy()

SIZES = list(range(6, 22, 3))
COLORS = Spectral5
N_SIZES = len(SIZES)
N_COLORS = len(COLORS)

# data cleanup
df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)
del df['name']

columns = sorted(df.columns)
discrete = [x for x in columns if df[x].dtype == object]
continuous = [x for x in columns if x not in discrete]

def create_figure():
    xs = df[x.value].values
    ys = df[y.value].values
    x_title = x.value.title()
    y_title = y.value.title()

    kw = dict()
    if x.value in discrete:
        kw['x_range'] = sorted(set(xs))
    if y.value in discrete:
        kw['y_range'] = sorted(set(ys))
    kw['title'] = "%s vs %s" % (x_title, y_title)

    p = figure(plot_height=600, plot_width=800, tools='pan,box_zoom,hover,reset', **kw)
    p.xaxis.axis_label = x_title
    p.yaxis.axis_label = y_title

    if x.value in discrete:
        p.xaxis.major_label_orientation = pd.np.pi / 4

    sz = 9
    if size.value != 'None':
        if len(set(df[size.value])) > N_SIZES:
            groups = pd.qcut(df[size.value].values, N_SIZES, duplicates='drop')
        else:
            groups = pd.Categorical(df[size.value])
        sz = [SIZES[xx] for xx in groups.codes]

    c = "#31AADE"
    if color.value != 'None':
        if len(set(df[color.value])) > N_COLORS:
            groups = pd.qcut(df[color.value].values, N_COLORS, duplicates='drop')
        else:
            groups = pd.Categorical(df[color.value])
        c = [COLORS[xx] for xx in groups.codes]

    p.circle(x=xs, y=ys, color=c, size=sz, line_color="white", alpha=0.6, hover_color='white', hover_alpha=0.5)

    return p

#def update(attr, old, new):
#    layout.children[1] = create_figure()
    
callback = CustomJS(args=dict(source=source), code="""
     var data = source.data;
     var f = cb_obj.value
     var x = data['x']
     var y = data['y']
     for (var i = 0; i < x.length; i++) {
         y[i] = Math.pow(x[i], f)
     }
     source.change.emit();
 """)


x = Select(title='X-Axis', value='mpg', options=columns)
#x.on_change('value', update)
x.js_on_change('value', callback)

y = Select(title='Y-Axis', value='hp', options=columns)
#y.on_change('value', update)

size = Select(title='Size', value='None', options=['None'] + continuous)
#size.on_change('value', update)
size.js_on_change('value', callback)

color = Select(title='Color', value='None', options=['None'] + continuous)
#color.on_change('value', update)
color.js_on_change('value', callback)

controls = column(x, y, color, size, width=100)
layout = row(controls, create_figure())

# slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
# slider.js_on_change('value', callback)


#doc.add_root(layout)
#doc.title = "Crossfilter"

# doc.theme = Theme(json=yaml.load("""
# attrs:
#     Figure:
#         background_fill_color: "#DDDDDD"
#         outline_line_color: white
#         toolbar_location: above
#         height: 500
#         width: 800
#     Grid:
#         grid_line_dash: [6, 4]
#         grid_line_color: white
# """, Loader=yaml.FullLoader))
    

In [15]:
show(layout)